In [52]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from collections import namedtuple

In [53]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
website = 'https://desaparecidos.policiacivil.mg.gov.br/desaparecido/album'
driver.get(website)

driver.maximize_window()

In [54]:
links = []
nomes = []
datas_desaparecimento = []
idades = []
locais_desaparecimento = []
teste = ['https://desaparecidos.policiacivil.mg.gov.br/desaparecido/exibir/2653', 'https://desaparecidos.policiacivil.mg.gov.br/desaparecido/exibir/2652']
pessoas = []

In [55]:
grid_div = driver.find_element(By.XPATH, "//div[@class='grid']")

links_elements = grid_div.find_elements(By.TAG_NAME, 'a')

for link in links_elements:
    href = link.get_attribute('href')
    if href:  
        links.append(href)

In [56]:
print(links)
len(links)

['https://desaparecidos.policiacivil.mg.gov.br/desaparecido/exibir/2653', 'https://desaparecidos.policiacivil.mg.gov.br/desaparecido/exibir/2652', 'https://desaparecidos.policiacivil.mg.gov.br/desaparecido/exibir/2651', 'https://desaparecidos.policiacivil.mg.gov.br/desaparecido/exibir/2650', 'https://desaparecidos.policiacivil.mg.gov.br/desaparecido/exibir/2649', 'https://desaparecidos.policiacivil.mg.gov.br/desaparecido/exibir/2648', 'https://desaparecidos.policiacivil.mg.gov.br/desaparecido/exibir/2645', 'https://desaparecidos.policiacivil.mg.gov.br/desaparecido/exibir/2644', 'https://desaparecidos.policiacivil.mg.gov.br/desaparecido/exibir/2643', 'https://desaparecidos.policiacivil.mg.gov.br/desaparecido/exibir/2642', 'https://desaparecidos.policiacivil.mg.gov.br/desaparecido/exibir/2641', 'https://desaparecidos.policiacivil.mg.gov.br/desaparecido/exibir/2640', 'https://desaparecidos.policiacivil.mg.gov.br/desaparecido/exibir/2639', 'https://desaparecidos.policiacivil.mg.gov.br/desa

742

In [57]:
pessoa_desaparecida = namedtuple('pessoa_desaparecida', ['nome', 'data_desaparecimento', 'idade', 'local_desaparecimento'])

In [62]:
for link in links:
    driver.get(link)

    # Usando try-except para lidar com possíveis erros se um elemento não for encontrado
    try:
        nomi = ''
        data_desaparecimento = ''
        idade = ''
        local_desaparecimento = ''

        # Coletando o nome, que é o texto do elemento <h1> dentro de <dd>
        nome = driver.find_element(By.CSS_SELECTOR, 'dl dd h1').text
        nomi = nome
        nomes.append(nome if nome else 'No info')
    except Exception as e:
        print(f"Erro ao coletar o nome: {e}")
        nomes.append('Não encontrado')

    # Localizando todos os elementos <dt> para identificar os campos corretos
    dts = driver.find_elements(By.CSS_SELECTOR, 'dl dt')

    # Iterando sobre os elementos <dt> e coletando o <dd> correspondente
    for dt in dts:
        dt_text = dt.text.strip().lower()
        try:
            next_dd = dt.find_element(By.XPATH, 'following-sibling::dd')
            dd_text = next_dd.text.strip()
            if 'data do desaparecimento' in dt_text:
                datas_desaparecimento.append(dd_text if dd_text else 'No info')
                data_desaparecimento = dd_text
            elif 'idade na ocasião do desaparecimento' in dt_text:
                idades.append(dd_text if dd_text else 'No info')
                idade = dd_text
            elif 'município/cidade do desaparecimento' in dt_text:
                locais_desaparecimento.append(dd_text if dd_text else 'No info')
                local_desaparecimento = dd_text


        except Exception as e:
            print(f"Erro ao coletar informações de {dt_text}: {e}")
    pessoa = pessoa_desaparecida(nomi,data_desaparecimento,idade, local_desaparecimento)
    pessoas.append(pessoa)

In [73]:
df_pessoas_desaparecidas = pd.DataFrame(pessoas)
df_pessoas_desaparecidas.to_csv('Desaparecidos MG.csv', index=False)